In [ ]:

from bqplot import pyplot as plt
import ipywidgets as widgets
import numpy as np

In [ ]:
# generate some fake 
n = 2000
x = np.linspace(0.0, 10.0, n)
np.random.seed(0)
y = np.cumsum(np.random.randn(n)*10).astype(int)

In [ ]:
fig_hist = plt.figure( title='Histogram')
hist = plt.hist(y, bins=25)

In [ ]:
hist.bins = 10;


In [ ]:
slider = widgets.IntSlider(description='Bins number', min=1, max=100, v_model=30)


In [ ]:
widgets.link((hist, 'bins'), (slider, 'value'))

fig_lines = plt.figure( title='Line Chart')
lines = plt.plot(x, y)

fig_lines.layout.width = 'auto'
fig_lines.layout.height = 'auto'
fig_hist.layout.width = 'auto'
fig_hist.layout.height = 'auto'

grid_layout = widgets.GridspecLayout(5, 3)

grid_layout[:2, :] = fig_lines
grid_layout[2:4, :] = fig_hist
grid_layout[4, 1] = slider

grid_layout.layout.height = '1000px'

grid_layout

In [ ]:
selector = plt.brush_int_selector()
def update_range(*ignore):
    if selector.selected is not None and len(selector.selected) == 2:
        xmin, xmax = selector.selected
        mask = (x > xmin) & (x < xmax)
        hist.sample = y[mask]
selector.observe(update_range, 'selected')